In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\users\sahil naidu\anaconda3\lib\site-packages (2.15.0)



In [7]:
pip install keras

In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment (3).csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)





291/291 - 22s - loss: 0.8211 - accuracy: 0.6472 - 22s/epoch - 75ms/step
144/144 - 2s - loss: 0.7511 - accuracy: 0.6782 - 2s/epoch - 14ms/step
0.7511035799980164
0.6782437562942505
['loss', 'accuracy']


In [4]:
model.save("sentiment_model.h5")

C:\Users\SAHIL NAIDU\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
pip install tweepy


     ---------------------------------------- 98.5/98.5 kB 2.8 MB/s eta 0:00:00


In [8]:
import tweepy
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

# Load the saved model
model = load_model("sentiment_model.h5")

# Define a function for preprocessing text
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-z0-9\s]', '', text)
    return text

# Example new text data
new_text = "A lot of good things are happening. We are respected again throughout the world, and that's a great thing. @realDonaldTrump"

# Preprocess the new text data
new_text = preprocess_text(new_text)

# Tokenize and pad the new text data
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts([new_text])
X_new = tokenizer.texts_to_sequences([new_text])
X_new = pad_sequences(X_new, maxlen=model.input_shape[1])

# Make predictions
predictions = model.predict(X_new)

# Determine the sentiment based on the prediction
sentiments = ['Negative', 'Neutral', 'Positive']
predicted_sentiment = sentiments[predictions.argmax()]

# Print the result
print("Predicted Sentiment: " + predicted_sentiment)


1/1 [==============================] - 1s 661ms/step
Predicted Sentiment: Negative


In [11]:
pip install scikeras

  Using cached scikeras-0.12.0-py3-none-any.whl (27 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
from scikeras.wrappers import KerasClassifier

In [13]:
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier

# Assuming the data loading and preprocessing steps are the same

max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
# Assuming tokenizer fitting and text preprocessing is done here

def createmodel(optimizer='adam'):
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Define the KerasClassifier with the build_fn as our model creation function
model = KerasClassifier(model=createmodel, verbose=2)

# Define hyperparameters to tune
param_grid = {
    'batch_size': [32, 64],
    'epochs': [1, 2],
    'optimizer': ['adam', 'rmsprop']
}

# Initialize GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
# Fit GridSearchCV
grid_result = grid.fit(X_train, Y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


194/194 - 13s - loss: 0.8572 - accuracy: 0.6320 - 13s/epoch - 65ms/step
97/97 - 1s - 997ms/epoch - 10ms/step
194/194 - 14s - loss: 0.8545 - accuracy: 0.6303 - 14s/epoch - 74ms/step
97/97 - 2s - 2s/epoch - 16ms/step
194/194 - 15s - loss: 0.8536 - accuracy: 0.6322 - 15s/epoch - 79ms/step
97/97 - 1s - 1s/epoch - 14ms/step
194/194 - 14s - loss: 0.8557 - accuracy: 0.6289 - 14s/epoch - 70ms/step
97/97 - 1s - 1s/epoch - 12ms/step
194/194 - 14s - loss: 0.8552 - accuracy: 0.6258 - 14s/epoch - 72ms/step
97/97 - 1s - 1s/epoch - 12ms/step
194/194 - 17s - loss: 0.8602 - accuracy: 0.6280 - 17s/epoch - 90ms/step
97/97 - 2s - 2s/epoch - 15ms/step
Epoch 1/2
194/194 - 15s - loss: 0.8596 - accuracy: 0.6299 - 15s/epoch - 75ms/step
Epoch 2/2
194/194 - 12s - loss: 0.7118 - accuracy: 0.6936 - 12s/epoch - 60ms/step
97/97 - 1s - 1s/epoch - 13ms/step
Epoch 1/2
194/194 - 15s - loss: 0.8635 - accuracy: 0.6329 - 15s/epoch - 76ms/step
Epoch 2/2
194/194 - 13s - loss: 0.7085 - accuracy: 0.6951 - 13s/epoch - 65ms/step